In [18]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
from functools import reduce
from sklearn.preprocessing import MinMaxScaler

In [19]:
pd.set_option('display.max_columns', 1000)

In [20]:
# Defining directories
dir_path = Path.cwd().parent
data_folder = dir_path.joinpath("data/processed")

#training frames
s1_path = data_folder.joinpath("sent1_train_season_medians_widened.parquet")
s2_path = data_folder.joinpath("sent2_train_season_medians_widened.parquet")
cov_path = data_folder.joinpath("cov_final.parquet")

model_df_path = data_folder.joinpath("model_df.parquet")
# species_model_df_path = data_folder.joinpath('species_train_df.csv')
species_dict_path = dir_path.joinpath("data/species_dict.json")

index_cols = ["long", "lat", 'year',"species_names"]


# previous model frame path 
actual_model_path = dir_path.joinpath("data/processed_532_features/species_train_df.csv")

In [21]:
s1 = pd.read_parquet(s1_path)
s1

,long,lat,year,species_names,vv_Descendingsummer,vv_Descendingwinter,vh_Descendingsummer,vh_Descendingwinter,vv_Ascendingsummer,vv_Ascendingwinter,vh_Ascendingsummer,vh_Ascendingwinter,VV_VH_Asummer,VV_VH_Awinter,VV_VH_Dsummer,VV_VH_Dwinter,VH_VV_Asummer,VH_VV_Awinter,VH_VV_Dsummer,VH_VV_Dwinter,SAR_NDVI_Asummer,SAR_NDVI_Awinter,SAR_NDVI_Dsummer,SAR_NDVI_Dwinter,DVI_Asummer,DVI_Awinter,DVI_Dsummer,DVI_Dwinter,SVI_Asummer,SVI_Awinter,SVI_Dsummer,SVI_Dwinter,RDVI_Asummer,RDVI_Awinter,RDVI_Dsummer,RDVI_Dwinter,NRDVI_Asummer,NRDVI_Awinter,NRDVI_Dsummer,NRDVI_Dwinter,SSDVI_Asummer,SSDVI_Awinter,SSDVI_Dsummer,SSDVI_Dwinter
0,75.66669580000007,32.25107380000003,2018,Bambusoideae spp.,22.552725,23.639657,19.700091,20.492180,23.384565,22.741578,19.956352,20.045554,1.179084,1.130519,1.156924,1.120752,0.848424,0.884551,0.865649,0.892258,-0.082086,-0.061261,-0.072357,-0.056938,-3.582735,-2.566007,-2.830202,-2.474472,43.769740,44.383245,41.528996,44.359597,-0.330660,-0.245967,-0.291276,-0.228494,-0.163029,-0.122064,-0.143803,-0.113509,-156.466199,-107.481875,-111.066850,-107.537649
1,75.66669580000007,32.25107380000003,2019,Bambusoideae spp.,23.856063,23.135292,19.956352,20.770598,21.846914,24.087430,20.530784,20.293079,1.064105,1.168957,1.198059,1.113785,0.939757,0.855464,0.834683,0.897885,-0.031057,-0.077898,-0.090106,-0.053817,-1.316130,-3.309932,-3.943802,-2.364695,42.433853,44.380509,43.768323,43.905890,-0.124348,-0.313493,-0.363376,-0.215900,-0.062054,-0.154856,-0.178761,-0.107319,-55.774555,-140.641534,-172.613601,-103.918188
2,75.66669580000007,32.25107380000003,2020,Bambusoideae spp.,23.268281,23.263359,21.227070,19.558451,22.068259,22.833012,20.086002,20.043214,1.104880,1.177081,1.196235,1.196378,0.905535,0.849559,0.836198,0.835856,-0.049694,-0.081339,-0.089273,-0.089410,-2.090337,-3.590219,-4.028260,-3.818532,42.235219,42.376016,43.871136,43.480976,-0.199344,-0.327523,-0.360037,-0.360521,-0.099106,-0.161609,-0.177096,-0.177402,-87.990981,-151.061373,-171.256533,-163.082568
3,75.66669580000007,32.25107380000003,2021,Bambusoideae spp.,24.927604,24.248816,19.777236,20.024829,22.430380,22.405492,20.293838,20.827854,1.136516,1.130413,1.218761,1.162297,0.882609,0.884633,0.820505,0.860365,-0.063077,-0.061215,-0.098596,-0.075058,-2.785947,-2.730013,-4.474374,-3.436265,42.724218,44.252407,44.478554,43.961993,-0.253907,-0.245780,-0.398256,-0.301932,-0.125368,-0.121973,-0.195294,-0.149274,-123.071242,-108.863796,-203.050827,-157.318045
4,75.66669580000007,32.25107380000003,2022,Bambusoideae spp.,24.149733,22.787536,20.453230,19.684829,23.242825,23.933544,20.487848,19.382821,1.120731,1.235955,1.189528,1.123760,0.892275,0.809453,0.840669,0.889870,-0.056929,-0.105405,-0.086561,-0.058274,-2.469331,-4.550723,-3.876463,-2.430380,44.256146,43.316365,44.533811,42.742463,-0.228456,-0.426502,-0.348859,-0.233890,-0.113490,-0.208427,-0.171835,-0.116153,-107.109181,-196.687173,-173.599348,-107.662974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893437,78.12624330000006,31.258713300000068,2019,Rhododendron arboreum,24.899585,21.450173,22.922561,18.317562,26.345883,26.085735,23.304138,23.273577,1.143444,1.127325,1.093749,1.171387,0.874551,0.897222,0.914405,0.853689,-0.066922,-0.056851,-0.044742,-0.078930,-3.337538,-2.812157,-2.190940,-3.046949,49.981772,49.422264,47.822146,38.482874,-0.268893,-0.230103,-0.179344,-0.317698,-0.133248,-0.112385,-0.089297,-0.156882,-165.262369,-139.104165,-94.545533,-136.152344
893438,78.12624330000006,31.258713300000068,2020,Rhododendron arboreum,21.172713,21.895621,18.450980,19.714503,22.845933,23.283796,21.205739,19.235085,1.062779,1.220015,1.130216,1.122397,0.942758,0.819662,0.884786,0.890951,-0.029935,-0.099105,-0.061128,-0.057669,-1.357677,-4.198946,-2.710668,-2.402251,43.843893,42.757340,39.746038,40.691233,-0.120022,-0.400352,-0.245430,-0.231446,-0.059729,-0.196282,-0.121801,-0.114956

In [22]:
s2 = pd.read_parquet(s2_path)
s2

,long,lat,year,species_names,B11summer,B11winter,B02summer,B02winter,B12summer,B12winter,B08summer,B08winter,B04summer,B04winter,B03summer,B03winter,NDVIsummer,NDVIwinter,ARVI2summer,ARVI2winter,BWDRVIsummer,BWDRVIwinter,CVIsummer,CVIwinter,CTVIsummer,CTVIwinter,EVI2summer,EVI2winter,GVMIsummer,GVMIwinter,MSVAIhypersummer,MSVAIhyperwinter,MTVI2summer,MTVI2winter,MNDVIsummer,MNDVIwinter,OSAVIsummer,OSAVIwinter,PVIsummer,PVIwinter,SARVIsummer,SARVIwinter,SLAVIsummer,SLAVIwinter,TSAVI2summer,TSAVI2winter,WDVIsummer,WDVIwinter,WDRVIsummer,WDRVIwinter
0,75.66669580000007,32.25107380000003,2018,Bambusoideae spp.,0.289700,0.24600,0.154400,0.13630,0.235600,0.18410,0.324200,0.34990,0.17640,0.14930,0.17720,0.15820,0.287273,0.411741,0.156109,0.301737,0.377670,0.439325,1.910389,2.151097,0.887284,0.954851,0.204080,0.300179,0.262364,0.387179,0.197548,0.289427,0.159463,0.258520,0.173049,0.324125,0.251835,0.361074,-0.776231,-0.750811,13.339193,21.943751,0.794514,1.078607,36.628039,18.783879,0.185947,0.240258,0.287273,0.411741
1,75.66669580000007,32.25107380000003,2019,Bambusoideae spp.,0.258850,0.25620,0.112800,0.14105,0.195400,0.18470,0.292300,0.38535,0.13280,0.15455,0.12965,0.16150,0.400403,0.427497,0.288472,0.320172,0.477402,0.465031,2.683532,2.293912,0.947785,0.963061,0.249000,0.328547,0.295781,0.406719,0.237530,0.317695,0.196948,0.283551,0.200573,0.352000,0.325497,0.382530,-0.807782,-0.715748,18.051811,34.403773,0.916986,1.136096,-36.203756,50.859088,0.192434,0.269128,0.400403,0.427497
2,75.66669580000007,32.25107380000003,2020,Bambusoideae spp.,0.276450,0.25100,0.144650,0.13035,0.216700,0.18535,0.345250,0.35955,0.16110,0.15440,0.16560,0.15675,0.360856,0.397863,0.242202,0.285500,0.407303,0.466377,2.025796,2.256043,0.927653,0.947438,0.264676,0.296029,0.305009,0.381638,0.255691,0.285919,0.223837,0.248274,0.226889,0.318541,0.318257,0.352093,-0.755411,-0.741267,170.718932,189.844088,0.911091,1.057878,140.143094,33.429727,0.224103,0.243441,0.360856,0.397863
3,75.66669580000007,32.25107380000003,2021,Bambusoideae spp.,0.292600,0.17570,0.150000,0.07620,0.239950,0.10050,0.319600,0.28050,0.18160,0.08550,0.17520,0.09650,0.275399,0.530091,0.142217,0.440206,0.361201,0.570220,1.895898,2.556117,0.880560,1.014934,0.196552,0.346411,0.234981,0.516937,0.190535,0.334204,0.152337,0.284346,0.142406,0.469517,0.242141,0.427112,-0.780781,-0.819454,12.514366,8.397010,0.758327,1.496774,55.341681,5.534139,0.183037,0.223724,0.275399,0.530091
4,75.66669580000007,32.25107380000003,2022,Bambusoideae spp.,0.307800,0.24770,0.153600,0.14870,0.234000,0.18220,0.325900,0.36890,0.19020,0.15840,0.17780,0.16700,0.270070,0.417861,0.135982,0.308897,0.365854,0.434734,1.957255,2.142705,0.877536,0.958051,0.195165,0.321921,0.268881,0.409546,0.189574,0.311482,0.145198,0.280120,0.182296,0.355349,0.238824,0.374587,-0.774549,-0.732019,13.678397,16.993831,0.789678,1.128400,123.528038,27.768672,0.185050,0.259860,0.270070,0.417861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893437,78.12624330000006,31.258713300000068,2019,Rhododendron arboreum,0.243600,0.16170,0.140525,0.11845,0.178200,0.13325,0.346600,0.20275,0.13695,0.12440,0.14730,0.12090,0.416348,0.239591,0.307127,0.100321,0.403482,0.263483,2.149144,1.731773,0.955703,0.859991,0.307733,0.130475,0.378621,0.312071,0.295786,0.122100,0.268972,0.094354,0.308246,0.184026,0.364786,0.186600,-0.754075,-0.896355,-4.395381,5.455463,1.089953,0.767643,25.292495,6.274160,0.243614,0.109201,0.416348,0.239591
893438,78.12624330000006,31.258713300000068,2020,Rhododendron arboreum,0.221750,0.15530,0.145950,0.15385,0.184700,0.13620,0.316400,0.19710,0.14870,0.13250,0.18420,0.14450,0.369943,0.220117,0.252834,0.077537,0.353816,0.152081,1.620617,1.446515,0.932707,0.848594,0.251145,0.118107,0.344915,0.314275,0.240886,0.110411,0.232998,0.097110,0.264865,0.186134,0.315786,0.170207,-0.783946,-0.901943,10.94

In [23]:
cov_df = pd.read_parquet(cov_path)
cov_df

,long,lat,species_names,sand_f,elevation,cfvo_f,phh2o_f,soc_f,slope,clay_f,nitrogen_f,cec_f,bdod_f,aspect,silt_f,ocd_f
0,75.66669580000007,32.25107380000003,Bambusoideae spp.,26.49250,352.0,13.5475,4.685,6.29375,13.422151,22.47375,8.6365,10.74375,1.008625,254.256500,26.03375,7.0275
1,75.66676390000003,32.25073910000003,Bambusoideae spp.,26.49250,350.0,13.5475,4.685,6.29375,10.147419,22.47375,8.6365,10.74375,1.008625,254.256454,26.03375,7.0275
2,75.66686810000004,32.25026770000005,Bambusoideae spp.,26.49250,349.0,13.5475,4.685,6.29375,8.110854,22.47375,8.6365,10.74375,1.008625,250.076767,26.03375,7.0275
3,75.66719240000003,32.249825100000066,Bambusoideae spp.,26.49250,351.0,13.5475,4.685,6.29375,9.826173,22.47375,8.6365,10.74375,1.008625,242.139969,26.03375,7.0275
4,75.66729090000007,32.24962370000003,Bambusoideae spp.,26.49250,348.0,13.5475,4.685,6.29375,6.125500,22.47375,8.6365,10.74375,1.008625,243.095016,26.03375,7.0275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148902,78.12497830000007,31.25865170000003,Rhododendron arboreum,36.73625,2309.0,19.9450,4.570,30.67875,54.879799,16.27250,11.7515,14.13875,0.968750,325.060272,21.98625,12.1675
148903,78.12522000000007,31.258880000000033,Rhododendron arboreum,36.73625,2292.0,19.9450,4.570,30.67875,58.185696,16.27250,11.7515,14.13875,0.968750,319.749359,21.98625,12.1675
148904,78.12614830000007,31.258660000000077,Rhododendron arboreum,36.73625,2361.0,19.9450,4.570,30.06625,30.714142,16.27250,11.7515,14.13875,0.966375,292.424896,21.98625,12.1675
148905,78.12618260000005,31.25882800000005,Rhododendron arboreum,36.73625,2350.0,19.9450,4.570,30.06625,43.765118,16.27250,11.7515,14.13875,0.966375,348.596802,21.98625,12.1675


In [24]:
sent_merged_df = reduce(lambda left, right: pd.merge(left, right, on=index_cols, how='outer', validate="1:1"), [s1, s2])
sent_merged_df



,long,lat,year,species_names,vv_Descendingsummer,vv_Descendingwinter,vh_Descendingsummer,vh_Descendingwinter,vv_Ascendingsummer,vv_Ascendingwinter,vh_Ascendingsummer,vh_Ascendingwinter,VV_VH_Asummer,VV_VH_Awinter,VV_VH_Dsummer,VV_VH_Dwinter,VH_VV_Asummer,VH_VV_Awinter,VH_VV_Dsummer,VH_VV_Dwinter,SAR_NDVI_Asummer,SAR_NDVI_Awinter,SAR_NDVI_Dsummer,SAR_NDVI_Dwinter,DVI_Asummer,DVI_Awinter,DVI_Dsummer,DVI_Dwinter,SVI_Asummer,SVI_Awinter,SVI_Dsummer,SVI_Dwinter,RDVI_Asummer,RDVI_Awinter,RDVI_Dsummer,RDVI_Dwinter,NRDVI_Asummer,NRDVI_Awinter,NRDVI_Dsummer,NRDVI_Dwinter,SSDVI_Asummer,SSDVI_Awinter,SSDVI_Dsummer,SSDVI_Dwinter,B11summer,B11winter,B02summer,B02winter,B12summer,B12winter,B08summer,B08winter,B04summer,B04winter,B03summer,B03winter,NDVIsummer,NDVIwinter,ARVI2summer,ARVI2winter,BWDRVIsummer,BWDRVIwinter,CVIsummer,CVIwinter,CTVIsummer,CTVIwinter,EVI2summer,EVI2winter,GVMIsummer,GVMIwinter,MSVAIhypersummer,MSVAIhyperwinter,MTVI2summer,MTVI2winter,MNDVIsummer,MNDVIwinter,OSAVIsummer,OSAVIwinter,PVIsummer,PVIwinter,SARVIsummer,SARVIwinter,SLAVIsummer,SLAVIwinter,TSAVI2summer,TSAVI2winter,WDVIsummer,WDVIwinter,WDRVIsummer,WDRVIwinter
0,75.66669580000007,32.25107380000003,2018,Bambusoideae spp.,22.552725,23.639657,19.700091,20.492180,23.384565,22.741578,19.956352,20.045554,1.179084,1.130519,1.156924,1.120752,0.848424,0.884551,0.865649,0.892258,-0.082086,-0.061261,-0.072357,-0.056938,-3.582735,-2.566007,-2.830202,-2.474472,43.769740,44.383245,41.528996,44.359597,-0.330660,-0.245967,-0.291276,-0.228494,-0.163029,-0.122064,-0.143803,-0.113509,-156.466199,-107.481875,-111.066850,-107.537649,0.289700,0.24600,0.154400,0.13630,0.235600,0.18410,0.324200,0.34990,0.17640,0.14930,0.17720,0.15820,0.287273,0.411741,0.156109,0.301737,0.377670,0.439325,1.910389,2.151097,0.887284,0.954851,0.204080,0.300179,0.262364,0.387179,0.197548,0.289427,0.159463,0.258520,0.173049,0.324125,0.251835,0.361074,-0.776231,-0.750811,13.339193,21.943751,0.794514,1.078607,36.628039,18.783879,0.185947,0.240258,0.287273,0.411741
1,75.66669580000007,32.25107380000003,2019,Bambusoideae spp.,23.856063,23.135292,19.956352,20.770598,21.846914,24.087430,20.530784,20.293079,1.064105,1.168957,1.198059,1.113785,0.939757,0.855464,0.834683,0.897885,-0.031057,-0.077898,-0.090106,-0.053817,-1.316130,-3.309932,-3.943802,-2.364695,42.433853,44.380509,43.768323,43.905890,-0.124348,-0.313493,-0.363376,-0.215900,-0.062054,-0.154856,-0.178761,-0.107319,-55.774555,-140.641534,-172.613601,-103.918188,0.258850,0.25620,0.112800,0.14105,0.195400,0.18470,0.292300,0.38535,0.13280,0.15455,0.12965,0.16150,0.400403,0.427497,0.288472,0.320172,0.477402,0.465031,2.683532,2.293912,0.947785,0.963061,0.249000,0.328547,0.295781,0.406719,0.237530,0.317695,0.196948,0.283551,0.200573,0.352000,0.325497,0.382530,-0.807782,-0.715748,18.051811,34.403773,0.916986,1.136096,-36.203756,50.859088,0.192434,0.269128,0.400403,0.427497
2,75.66669580000007,32.25107380000003,2020,Bambusoideae spp.,23.268281,23.263359,21.227070,19.558451,22.068259,22.833012,20.086002,20.043214,1.104880,1.177081,1.196235,1.196378,0.905535,0.849559,0.836198,0.835856,-0.049694,-0.081339,-0.089273,-0.089410,-2.090337,-3.590219,-4.028260,-3.818532,42.235219,42.376016,43.871136,43.480976,-0.199344,-0.327523,-0.360037,-0.360521,-0.099106,-0.161609,-0.177096,-0.177402,-87.990981,-151.061373,-171.256533,-163.082568,0.276450,0.25100,0.144650,0.13035,0.216700,0.18535,0.345250,0.35955,0.16110,0.15440,0.16560,0.15675,0.360856,0.397863,0.242202,0.285500,0.407303,0.466377,2.025796,2.256043,0.927653,0.947438,0.264676,0.296029,0.305009,0.381638,0.255691,0.285919,0.223837,0.248274,0.226889,0.318541,0.318257,0.352093,-0.755411,-0.741267,170.718932,189.844088,0.911091,1.057878,140.143094,33.429727,0.224103,0.243441,0.360856,0.397863
3,75.66669580000007,32.25107380000003,2021,Bambusoideae spp.,24.927604,24.248816,19.777236,20.024829,22.430380,22.405492,20.293838,20.827854,1.136516,1.130413,1.218761,1.162297,0.882609,0.884633,0.820505,0.860365,-0.063077,-0.0612

In [25]:
merged_df = reduce(lambda left, right: pd.merge(left, right,  on=["long", "lat", "species_names"], how='outer', validate="m:1"), [sent_merged_df, cov_df])
merged_df



,long,lat,year,species_names,vv_Descendingsummer,vv_Descendingwinter,vh_Descendingsummer,vh_Descendingwinter,vv_Ascendingsummer,vv_Ascendingwinter,vh_Ascendingsummer,vh_Ascendingwinter,VV_VH_Asummer,VV_VH_Awinter,VV_VH_Dsummer,VV_VH_Dwinter,VH_VV_Asummer,VH_VV_Awinter,VH_VV_Dsummer,VH_VV_Dwinter,SAR_NDVI_Asummer,SAR_NDVI_Awinter,SAR_NDVI_Dsummer,SAR_NDVI_Dwinter,DVI_Asummer,DVI_Awinter,DVI_Dsummer,DVI_Dwinter,SVI_Asummer,SVI_Awinter,SVI_Dsummer,SVI_Dwinter,RDVI_Asummer,RDVI_Awinter,RDVI_Dsummer,RDVI_Dwinter,NRDVI_Asummer,NRDVI_Awinter,NRDVI_Dsummer,NRDVI_Dwinter,SSDVI_Asummer,SSDVI_Awinter,SSDVI_Dsummer,SSDVI_Dwinter,B11summer,B11winter,B02summer,B02winter,B12summer,B12winter,B08summer,B08winter,B04summer,B04winter,B03summer,B03winter,NDVIsummer,NDVIwinter,ARVI2summer,ARVI2winter,BWDRVIsummer,BWDRVIwinter,CVIsummer,CVIwinter,CTVIsummer,CTVIwinter,EVI2summer,EVI2winter,GVMIsummer,GVMIwinter,MSVAIhypersummer,MSVAIhyperwinter,MTVI2summer,MTVI2winter,MNDVIsummer,MNDVIwinter,OSAVIsummer,OSAVIwinter,PVIsummer,PVIwinter,SARVIsummer,SARVIwinter,SLAVIsummer,SLAVIwinter,TSAVI2summer,TSAVI2winter,WDVIsummer,WDVIwinter,WDRVIsummer,WDRVIwinter,sand_f,elevation,cfvo_f,phh2o_f,soc_f,slope,clay_f,nitrogen_f,cec_f,bdod_f,aspect,silt_f,ocd_f
0,75.66669580000007,32.25107380000003,2018,Bambusoideae spp.,22.552725,23.639657,19.700091,20.492180,23.384565,22.741578,19.956352,20.045554,1.179084,1.130519,1.156924,1.120752,0.848424,0.884551,0.865649,0.892258,-0.082086,-0.061261,-0.072357,-0.056938,-3.582735,-2.566007,-2.830202,-2.474472,43.769740,44.383245,41.528996,44.359597,-0.330660,-0.245967,-0.291276,-0.228494,-0.163029,-0.122064,-0.143803,-0.113509,-156.466199,-107.481875,-111.066850,-107.537649,0.289700,0.24600,0.154400,0.13630,0.235600,0.18410,0.324200,0.34990,0.17640,0.14930,0.17720,0.15820,0.287273,0.411741,0.156109,0.301737,0.377670,0.439325,1.910389,2.151097,0.887284,0.954851,0.204080,0.300179,0.262364,0.387179,0.197548,0.289427,0.159463,0.258520,0.173049,0.324125,0.251835,0.361074,-0.776231,-0.750811,13.339193,21.943751,0.794514,1.078607,36.628039,18.783879,0.185947,0.240258,0.287273,0.411741,26.49250,352.0,13.5475,4.685,6.29375,13.422151,22.47375,8.6365,10.74375,1.008625,254.256500,26.03375,7.0275
1,75.66669580000007,32.25107380000003,2019,Bambusoideae spp.,23.856063,23.135292,19.956352,20.770598,21.846914,24.087430,20.530784,20.293079,1.064105,1.168957,1.198059,1.113785,0.939757,0.855464,0.834683,0.897885,-0.031057,-0.077898,-0.090106,-0.053817,-1.316130,-3.309932,-3.943802,-2.364695,42.433853,44.380509,43.768323,43.905890,-0.124348,-0.313493,-0.363376,-0.215900,-0.062054,-0.154856,-0.178761,-0.107319,-55.774555,-140.641534,-172.613601,-103.918188,0.258850,0.25620,0.112800,0.14105,0.195400,0.18470,0.292300,0.38535,0.13280,0.15455,0.12965,0.16150,0.400403,0.427497,0.288472,0.320172,0.477402,0.465031,2.683532,2.293912,0.947785,0.963061,0.249000,0.328547,0.295781,0.406719,0.237530,0.317695,0.196948,0.283551,0.200573,0.352000,0.325497,0.382530,-0.807782,-0.715748,18.051811,34.403773,0.916986,1.136096,-36.203756,50.859088,0.192434,0.269128,0.400403,0.427497,26.49250,352.0,13.5475,4.685,6.29375,13.422151,22.47375,8.6365,10.74375,1.008625,254.256500,26.03375,7.0275
2,75.66669580000007,32.25107380000003,2020,Bambusoideae spp.,23.268281,23.263359,21.227070,19.558451,22.068259,22.833012,20.086002,20.043214,1.104880,1.177081,1.196235,1.196378,0.905535,0.849559,0.836198,0.835856,-0.049694,-0.081339,-0.089273,-0.089410,-2.090337,-3.590219,-4.028260,-3.818532,42.235219,42.376016,43.871136,43.480976,-0.199344,-0.327523,-0.360037,-0.360521,-0.099106,-0.161609,-0.177096,-0.177402,-87.990981,-151.061373,-171.256533,-163.082568,0.276450,0.25100,0.144650,0.13035,0.216700,0.18535,0.345250,0.35955,0.16110,0.15440,0.16560,0.15675,0.360856,0.397863,0.242202,0.285500,0.407303,0.466377,2.025796,2.256043,0.927653,0.947438,0.264676,0.296029,0.305009,0.381638,0.255691,0.285919,0.223837,0.248274,0.226889,0.318541,0.318257,0.352093,-0.755411,-0.741267,170.718932,189.8

### Species Codes and JSON File

In [26]:
if not species_dict_path.exists():

    #exporting species dictionary as json
    cats = pd.Categorical(merged_df['species_names'])
    species_dict = dict(zip(range(len(cats.categories)), cats.categories))

    with open(species_dict_path, 'w') as json_file:
        json.dump(species_dict, json_file, indent=4)

In [27]:
merged_df["species"] = pd.Categorical(merged_df['species_names']).codes
merged_df.drop('species_names', axis=1, inplace=True)
merged_df

,long,lat,year,vv_Descendingsummer,vv_Descendingwinter,vh_Descendingsummer,vh_Descendingwinter,vv_Ascendingsummer,vv_Ascendingwinter,vh_Ascendingsummer,vh_Ascendingwinter,VV_VH_Asummer,VV_VH_Awinter,VV_VH_Dsummer,VV_VH_Dwinter,VH_VV_Asummer,VH_VV_Awinter,VH_VV_Dsummer,VH_VV_Dwinter,SAR_NDVI_Asummer,SAR_NDVI_Awinter,SAR_NDVI_Dsummer,SAR_NDVI_Dwinter,DVI_Asummer,DVI_Awinter,DVI_Dsummer,DVI_Dwinter,SVI_Asummer,SVI_Awinter,SVI_Dsummer,SVI_Dwinter,RDVI_Asummer,RDVI_Awinter,RDVI_Dsummer,RDVI_Dwinter,NRDVI_Asummer,NRDVI_Awinter,NRDVI_Dsummer,NRDVI_Dwinter,SSDVI_Asummer,SSDVI_Awinter,SSDVI_Dsummer,SSDVI_Dwinter,B11summer,B11winter,B02summer,B02winter,B12summer,B12winter,B08summer,B08winter,B04summer,B04winter,B03summer,B03winter,NDVIsummer,NDVIwinter,ARVI2summer,ARVI2winter,BWDRVIsummer,BWDRVIwinter,CVIsummer,CVIwinter,CTVIsummer,CTVIwinter,EVI2summer,EVI2winter,GVMIsummer,GVMIwinter,MSVAIhypersummer,MSVAIhyperwinter,MTVI2summer,MTVI2winter,MNDVIsummer,MNDVIwinter,OSAVIsummer,OSAVIwinter,PVIsummer,PVIwinter,SARVIsummer,SARVIwinter,SLAVIsummer,SLAVIwinter,TSAVI2summer,TSAVI2winter,WDVIsummer,WDVIwinter,WDRVIsummer,WDRVIwinter,sand_f,elevation,cfvo_f,phh2o_f,soc_f,slope,clay_f,nitrogen_f,cec_f,bdod_f,aspect,silt_f,ocd_f,species
0,75.66669580000007,32.25107380000003,2018,22.552725,23.639657,19.700091,20.492180,23.384565,22.741578,19.956352,20.045554,1.179084,1.130519,1.156924,1.120752,0.848424,0.884551,0.865649,0.892258,-0.082086,-0.061261,-0.072357,-0.056938,-3.582735,-2.566007,-2.830202,-2.474472,43.769740,44.383245,41.528996,44.359597,-0.330660,-0.245967,-0.291276,-0.228494,-0.163029,-0.122064,-0.143803,-0.113509,-156.466199,-107.481875,-111.066850,-107.537649,0.289700,0.24600,0.154400,0.13630,0.235600,0.18410,0.324200,0.34990,0.17640,0.14930,0.17720,0.15820,0.287273,0.411741,0.156109,0.301737,0.377670,0.439325,1.910389,2.151097,0.887284,0.954851,0.204080,0.300179,0.262364,0.387179,0.197548,0.289427,0.159463,0.258520,0.173049,0.324125,0.251835,0.361074,-0.776231,-0.750811,13.339193,21.943751,0.794514,1.078607,36.628039,18.783879,0.185947,0.240258,0.287273,0.411741,26.49250,352.0,13.5475,4.685,6.29375,13.422151,22.47375,8.6365,10.74375,1.008625,254.256500,26.03375,7.0275,1
1,75.66669580000007,32.25107380000003,2019,23.856063,23.135292,19.956352,20.770598,21.846914,24.087430,20.530784,20.293079,1.064105,1.168957,1.198059,1.113785,0.939757,0.855464,0.834683,0.897885,-0.031057,-0.077898,-0.090106,-0.053817,-1.316130,-3.309932,-3.943802,-2.364695,42.433853,44.380509,43.768323,43.905890,-0.124348,-0.313493,-0.363376,-0.215900,-0.062054,-0.154856,-0.178761,-0.107319,-55.774555,-140.641534,-172.613601,-103.918188,0.258850,0.25620,0.112800,0.14105,0.195400,0.18470,0.292300,0.38535,0.13280,0.15455,0.12965,0.16150,0.400403,0.427497,0.288472,0.320172,0.477402,0.465031,2.683532,2.293912,0.947785,0.963061,0.249000,0.328547,0.295781,0.406719,0.237530,0.317695,0.196948,0.283551,0.200573,0.352000,0.325497,0.382530,-0.807782,-0.715748,18.051811,34.403773,0.916986,1.136096,-36.203756,50.859088,0.192434,0.269128,0.400403,0.427497,26.49250,352.0,13.5475,4.685,6.29375,13.422151,22.47375,8.6365,10.74375,1.008625,254.256500,26.03375,7.0275,1
2,75.66669580000007,32.25107380000003,2020,23.268281,23.263359,21.227070,19.558451,22.068259,22.833012,20.086002,20.043214,1.104880,1.177081,1.196235,1.196378,0.905535,0.849559,0.836198,0.835856,-0.049694,-0.081339,-0.089273,-0.089410,-2.090337,-3.590219,-4.028260,-3.818532,42.235219,42.376016,43.871136,43.480976,-0.199344,-0.327523,-0.360037,-0.360521,-0.099106,-0.161609,-0.177096,-0.177402,-87.990981,-151.061373,-171.256533,-163.082568,0.276450,0.25100,0.144650,0.13035,0.216700,0.18535,0.345250,0.35955,0.16110,0.15440,0.16560,0.15675,0.360856,0.397863,0.242202,0.285500,0.407303,0.466377,2.025796,2.256043,0.927653,0.947438,0.264676,0.296029,0.305009,0.381638,0.255691,0.285919,0.223837,0.248274,0.226889,0.318541,0.318257,0.352093,-0.755411,-0.741267,170.718932,189.844088,0.911091,1.057878,140.143094,33.429727,0.224103,0.

In [28]:
merged_df['species'].unique()

array([ 1,  4, 11,  6,  5,  7,  8,  3, 13,  2,  0, 10, 12,  9], dtype=int8)

### Normalizing

In [29]:
df_index_cols = merged_df[['long', 'lat', 'year','species']]
df_x = merged_df.drop(['long', 'lat', 'year','species'], axis=1)

df_x

,vv_Descendingsummer,vv_Descendingwinter,vh_Descendingsummer,vh_Descendingwinter,vv_Ascendingsummer,vv_Ascendingwinter,vh_Ascendingsummer,vh_Ascendingwinter,VV_VH_Asummer,VV_VH_Awinter,VV_VH_Dsummer,VV_VH_Dwinter,VH_VV_Asummer,VH_VV_Awinter,VH_VV_Dsummer,VH_VV_Dwinter,SAR_NDVI_Asummer,SAR_NDVI_Awinter,SAR_NDVI_Dsummer,SAR_NDVI_Dwinter,DVI_Asummer,DVI_Awinter,DVI_Dsummer,DVI_Dwinter,SVI_Asummer,SVI_Awinter,SVI_Dsummer,SVI_Dwinter,RDVI_Asummer,RDVI_Awinter,RDVI_Dsummer,RDVI_Dwinter,NRDVI_Asummer,NRDVI_Awinter,NRDVI_Dsummer,NRDVI_Dwinter,SSDVI_Asummer,SSDVI_Awinter,SSDVI_Dsummer,SSDVI_Dwinter,B11summer,B11winter,B02summer,B02winter,B12summer,B12winter,B08summer,B08winter,B04summer,B04winter,B03summer,B03winter,NDVIsummer,NDVIwinter,ARVI2summer,ARVI2winter,BWDRVIsummer,BWDRVIwinter,CVIsummer,CVIwinter,CTVIsummer,CTVIwinter,EVI2summer,EVI2winter,GVMIsummer,GVMIwinter,MSVAIhypersummer,MSVAIhyperwinter,MTVI2summer,MTVI2winter,MNDVIsummer,MNDVIwinter,OSAVIsummer,OSAVIwinter,PVIsummer,PVIwinter,SARVIsummer,SARVIwinter,SLAVIsummer,SLAVIwinter,TSAVI2summer,TSAVI2winter,WDVIsummer,WDVIwinter,WDRVIsummer,WDRVIwinter,sand_f,elevation,cfvo_f,phh2o_f,soc_f,slope,clay_f,nitrogen_f,cec_f,bdod_f,aspect,silt_f,ocd_f
0,22.552725,23.639657,19.700091,20.492180,23.384565,22.741578,19.956352,20.045554,1.179084,1.130519,1.156924,1.120752,0.848424,0.884551,0.865649,0.892258,-0.082086,-0.061261,-0.072357,-0.056938,-3.582735,-2.566007,-2.830202,-2.474472,43.769740,44.383245,41.528996,44.359597,-0.330660,-0.245967,-0.291276,-0.228494,-0.163029,-0.122064,-0.143803,-0.113509,-156.466199,-107.481875,-111.066850,-107.537649,0.289700,0.24600,0.154400,0.13630,0.235600,0.18410,0.324200,0.34990,0.17640,0.14930,0.17720,0.15820,0.287273,0.411741,0.156109,0.301737,0.377670,0.439325,1.910389,2.151097,0.887284,0.954851,0.204080,0.300179,0.262364,0.387179,0.197548,0.289427,0.159463,0.258520,0.173049,0.324125,0.251835,0.361074,-0.776231,-0.750811,13.339193,21.943751,0.794514,1.078607,36.628039,18.783879,0.185947,0.240258,0.287273,0.411741,26.49250,352.0,13.5475,4.685,6.29375,13.422151,22.47375,8.6365,10.74375,1.008625,254.256500,26.03375,7.0275
1,23.856063,23.135292,19.956352,20.770598,21.846914,24.087430,20.530784,20.293079,1.064105,1.168957,1.198059,1.113785,0.939757,0.855464,0.834683,0.897885,-0.031057,-0.077898,-0.090106,-0.053817,-1.316130,-3.309932,-3.943802,-2.364695,42.433853,44.380509,43.768323,43.905890,-0.124348,-0.313493,-0.363376,-0.215900,-0.062054,-0.154856,-0.178761,-0.107319,-55.774555,-140.641534,-172.613601,-103.918188,0.258850,0.25620,0.112800,0.14105,0.195400,0.18470,0.292300,0.38535,0.13280,0.15455,0.12965,0.16150,0.400403,0.427497,0.288472,0.320172,0.477402,0.465031,2.683532,2.293912,0.947785,0.963061,0.249000,0.328547,0.295781,0.406719,0.237530,0.317695,0.196948,0.283551,0.200573,0.352000,0.325497,0.382530,-0.807782,-0.715748,18.051811,34.403773,0.916986,1.136096,-36.203756,50.859088,0.192434,0.269128,0.400403,0.427497,26.49250,352.0,13.5475,4.685,6.29375,13.422151,22.47375,8.6365,10.74375,1.008625,254.256500,26.03375,7.0275
2,23.268281,23.263359,21.227070,19.558451,22.068259,22.833012,20.086002,20.043214,1.104880,1.177081,1.196235,1.196378,0.905535,0.849559,0.836198,0.835856,-0.049694,-0.081339,-0.089273,-0.089410,-2.090337,-3.590219,-4.028260,-3.818532,42.235219,42.376016,43.871136,43.480976,-0.199344,-0.327523,-0.360037,-0.360521,-0.099106,-0.161609,-0.177096,-0.177402,-87.990981,-151.061373,-171.256533,-163.082568,0.276450,0.25100,0.144650,0.13035,0.216700,0.18535,0.345250,0.35955,0.16110,0.15440,0.16560,0.15675,0.360856,0.397863,0.242202,0.285500,0.407303,0.466377,2.025796,2.256043,0.927653,0.947438,0.264676,0.296029,0.305009,0.381638,0.255691,0.285919,0.223837,0.248274,0.226889,0.318541,0.318257,0.352093,-0.755411,-0.741267,170.718932,189.844088,0.911091,1.057878,140.143094,33.429727,0.224103,0.243441,0.360856,0.397863,26.49250,352.0,13.5475,4.685,6.29375,13.422151,22.47375,8.6365,10.74375,1.008625,254.256500,26.03375,7.0275
3,24.927604,24.2

In [30]:
min_max_scaler = MinMaxScaler()

df_scaled = pd.DataFrame(min_max_scaler.fit_transform(df_x.values), columns=df_x.columns)
df_scaled

,vv_Descendingsummer,vv_Descendingwinter,vh_Descendingsummer,vh_Descendingwinter,vv_Ascendingsummer,vv_Ascendingwinter,vh_Ascendingsummer,vh_Ascendingwinter,VV_VH_Asummer,VV_VH_Awinter,VV_VH_Dsummer,VV_VH_Dwinter,VH_VV_Asummer,VH_VV_Awinter,VH_VV_Dsummer,VH_VV_Dwinter,SAR_NDVI_Asummer,SAR_NDVI_Awinter,SAR_NDVI_Dsummer,SAR_NDVI_Dwinter,DVI_Asummer,DVI_Awinter,DVI_Dsummer,DVI_Dwinter,SVI_Asummer,SVI_Awinter,SVI_Dsummer,SVI_Dwinter,RDVI_Asummer,RDVI_Awinter,RDVI_Dsummer,RDVI_Dwinter,NRDVI_Asummer,NRDVI_Awinter,NRDVI_Dsummer,NRDVI_Dwinter,SSDVI_Asummer,SSDVI_Awinter,SSDVI_Dsummer,SSDVI_Dwinter,B11summer,B11winter,B02summer,B02winter,B12summer,B12winter,B08summer,B08winter,B04summer,B04winter,B03summer,B03winter,NDVIsummer,NDVIwinter,ARVI2summer,ARVI2winter,BWDRVIsummer,BWDRVIwinter,CVIsummer,CVIwinter,CTVIsummer,CTVIwinter,EVI2summer,EVI2winter,GVMIsummer,GVMIwinter,MSVAIhypersummer,MSVAIhyperwinter,MTVI2summer,MTVI2winter,MNDVIsummer,MNDVIwinter,OSAVIsummer,OSAVIwinter,PVIsummer,PVIwinter,SARVIsummer,SARVIwinter,SLAVIsummer,SLAVIwinter,TSAVI2summer,TSAVI2winter,WDVIsummer,WDVIwinter,WDRVIsummer,WDRVIwinter,sand_f,elevation,cfvo_f,phh2o_f,soc_f,slope,clay_f,nitrogen_f,cec_f,bdod_f,aspect,silt_f,ocd_f
0,0.396606,0.471055,0.416042,0.470285,0.451115,0.404824,0.341066,0.350320,0.338847,0.269886,0.336448,0.307829,0.519535,0.602601,0.497106,0.506712,0.584585,0.662452,0.574248,0.594059,0.621240,0.716986,0.636023,0.628458,0.392045,0.417418,0.344660,0.484055,0.605552,0.680172,0.596939,0.617232,0.564211,0.645100,0.552461,0.572055,0.664566,0.754933,0.690628,0.665937,0.313081,0.228517,0.162219,0.118167,0.317432,0.262492,0.310292,0.278019,0.190256,0.129642,0.190458,0.140687,0.433215,0.551315,0.433215,0.551315,0.487001,0.646174,0.005272,0.280977,0.526269,0.633711,0.382663,0.421141,0.431660,0.561714,0.408945,0.475636,0.363129,0.447514,0.412795,0.542756,0.493089,0.560768,0.310292,0.278019,1.0,1.0,0.114210,0.146335,0.461660,0.460997,0.303479,0.271846,0.433215,0.551315,0.19650,0.024909,0.319782,0.469907,0.040873,0.197817,0.585997,0.088080,0.101551,0.636993,0.708511,0.576992,0.118323
1,0.460550,0.446541,0.429824,0.487266,0.366455,0.478185,0.370159,0.362405,0.183898,0.322430,0.383513,0.300665,0.709946,0.540842,0.444008,0.515246,0.761503,0.603882,0.522477,0.602234,0.788002,0.664917,0.574423,0.633923,0.355852,0.417348,0.407194,0.471022,0.774423,0.624019,0.547459,0.624950,0.748836,0.584238,0.498555,0.580663,0.808862,0.710048,0.625380,0.669283,0.274354,0.238709,0.117719,0.122591,0.257691,0.263396,0.272727,0.308818,0.142307,0.134439,0.137201,0.143900,0.533531,0.565906,0.533531,0.565906,0.580706,0.665711,0.008468,0.302683,0.619712,0.646971,0.433430,0.450585,0.473029,0.584626,0.457725,0.509849,0.404839,0.477641,0.441885,0.570223,0.577584,0.584228,0.272727,0.308818,1.0,1.0,0.150823,0.158899,0.461616,0.461170,0.312718,0.302638,0.533531,0.565906,0.19650,0.024909,0.319782,0.469907,0.040873,0.197817,0.585997,0.088080,0.101551,0.636993,0.708511,0.576992,0.118323
2,0.431712,0.452766,0.498169,0.413335,0.378641,0.409808,0.347632,0.350206,0.238847,0.333536,0.381426,0.385602,0.638601,0.528305,0.446605,0.421169,0.696887,0.591767,0.524909,0.509017,0.731041,0.645299,0.569751,0.561549,0.350470,0.365744,0.410065,0.458816,0.713037,0.612352,0.549750,0.536319,0.681090,0.571704,0.501122,0.483209,0.762694,0.695944,0.626819,0.614591,0.296447,0.233513,0.151789,0.112627,0.289345,0.264374,0.335080,0.286403,0.173430,0.134302,0.177465,0.139276,0.498464,0.538463,0.498464,0.538463,0.514843,0.666735,0.005749,0.296927,0.588618,0.621737,0.451147,0.416834,0.484453,0.555216,0.479883,0.471391,0.434758,0.435182,0.469696,0.537254,0.569279,0.550948,0.335080,0.286403,1.0,1.0,0.149061,0.141805,0.461722,0.461076,0.357819,0.275241,0.498464,0.538463,0.19650,0.024909,0.319782,0.469907,0.040873,0.197817,0.585997,0.088080,0.101551,0.636993,0.708511,0.576992,0.118323
3,0.513121,0.500663,0.420191,0.441780,0.398579,0.386505,0.358158,0.388514,0.281481,0.269742,0.407200,0.350554,0.590805,0.602773,0.419696,0

### Model Data Frame

In [31]:
model_df = pd.concat([df_index_cols, df_scaled], axis=1)
model_df

,long,lat,year,species,vv_Descendingsummer,vv_Descendingwinter,vh_Descendingsummer,vh_Descendingwinter,vv_Ascendingsummer,vv_Ascendingwinter,vh_Ascendingsummer,vh_Ascendingwinter,VV_VH_Asummer,VV_VH_Awinter,VV_VH_Dsummer,VV_VH_Dwinter,VH_VV_Asummer,VH_VV_Awinter,VH_VV_Dsummer,VH_VV_Dwinter,SAR_NDVI_Asummer,SAR_NDVI_Awinter,SAR_NDVI_Dsummer,SAR_NDVI_Dwinter,DVI_Asummer,DVI_Awinter,DVI_Dsummer,DVI_Dwinter,SVI_Asummer,SVI_Awinter,SVI_Dsummer,SVI_Dwinter,RDVI_Asummer,RDVI_Awinter,RDVI_Dsummer,RDVI_Dwinter,NRDVI_Asummer,NRDVI_Awinter,NRDVI_Dsummer,NRDVI_Dwinter,SSDVI_Asummer,SSDVI_Awinter,SSDVI_Dsummer,SSDVI_Dwinter,B11summer,B11winter,B02summer,B02winter,B12summer,B12winter,B08summer,B08winter,B04summer,B04winter,B03summer,B03winter,NDVIsummer,NDVIwinter,ARVI2summer,ARVI2winter,BWDRVIsummer,BWDRVIwinter,CVIsummer,CVIwinter,CTVIsummer,CTVIwinter,EVI2summer,EVI2winter,GVMIsummer,GVMIwinter,MSVAIhypersummer,MSVAIhyperwinter,MTVI2summer,MTVI2winter,MNDVIsummer,MNDVIwinter,OSAVIsummer,OSAVIwinter,PVIsummer,PVIwinter,SARVIsummer,SARVIwinter,SLAVIsummer,SLAVIwinter,TSAVI2summer,TSAVI2winter,WDVIsummer,WDVIwinter,WDRVIsummer,WDRVIwinter,sand_f,elevation,cfvo_f,phh2o_f,soc_f,slope,clay_f,nitrogen_f,cec_f,bdod_f,aspect,silt_f,ocd_f
0,75.66669580000007,32.25107380000003,2018,1,0.396606,0.471055,0.416042,0.470285,0.451115,0.404824,0.341066,0.350320,0.338847,0.269886,0.336448,0.307829,0.519535,0.602601,0.497106,0.506712,0.584585,0.662452,0.574248,0.594059,0.621240,0.716986,0.636023,0.628458,0.392045,0.417418,0.344660,0.484055,0.605552,0.680172,0.596939,0.617232,0.564211,0.645100,0.552461,0.572055,0.664566,0.754933,0.690628,0.665937,0.313081,0.228517,0.162219,0.118167,0.317432,0.262492,0.310292,0.278019,0.190256,0.129642,0.190458,0.140687,0.433215,0.551315,0.433215,0.551315,0.487001,0.646174,0.005272,0.280977,0.526269,0.633711,0.382663,0.421141,0.431660,0.561714,0.408945,0.475636,0.363129,0.447514,0.412795,0.542756,0.493089,0.560768,0.310292,0.278019,1.0,1.0,0.114210,0.146335,0.461660,0.460997,0.303479,0.271846,0.433215,0.551315,0.19650,0.024909,0.319782,0.469907,0.040873,0.197817,0.585997,0.088080,0.101551,0.636993,0.708511,0.576992,0.118323
1,75.66669580000007,32.25107380000003,2019,1,0.460550,0.446541,0.429824,0.487266,0.366455,0.478185,0.370159,0.362405,0.183898,0.322430,0.383513,0.300665,0.709946,0.540842,0.444008,0.515246,0.761503,0.603882,0.522477,0.602234,0.788002,0.664917,0.574423,0.633923,0.355852,0.417348,0.407194,0.471022,0.774423,0.624019,0.547459,0.624950,0.748836,0.584238,0.498555,0.580663,0.808862,0.710048,0.625380,0.669283,0.274354,0.238709,0.117719,0.122591,0.257691,0.263396,0.272727,0.308818,0.142307,0.134439,0.137201,0.143900,0.533531,0.565906,0.533531,0.565906,0.580706,0.665711,0.008468,0.302683,0.619712,0.646971,0.433430,0.450585,0.473029,0.584626,0.457725,0.509849,0.404839,0.477641,0.441885,0.570223,0.577584,0.584228,0.272727,0.308818,1.0,1.0,0.150823,0.158899,0.461616,0.461170,0.312718,0.302638,0.533531,0.565906,0.19650,0.024909,0.319782,0.469907,0.040873,0.197817,0.585997,0.088080,0.101551,0.636993,0.708511,0.576992,0.118323
2,75.66669580000007,32.25107380000003,2020,1,0.431712,0.452766,0.498169,0.413335,0.378641,0.409808,0.347632,0.350206,0.238847,0.333536,0.381426,0.385602,0.638601,0.528305,0.446605,0.421169,0.696887,0.591767,0.524909,0.509017,0.731041,0.645299,0.569751,0.561549,0.350470,0.365744,0.410065,0.458816,0.713037,0.612352,0.549750,0.536319,0.681090,0.571704,0.501122,0.483209,0.762694,0.695944,0.626819,0.614591,0.296447,0.233513,0.151789,0.112627,0.289345,0.264374,0.335080,0.286403,0.173430,0.134302,0.177465,0.139276,0.498464,0.538463,0.498464,0.538463,0.514843,0.666735,0.005749,0.296927,0.588618,0.621737,0.451147,0.416834,0.484453,0.555216,0.479883,0.471391,0.434758,0.435182,0.469696,0.537254,0.569279,0.550948,0.335080,0.286403,1.0,1.0,0.149061,0.141805,0.461722,0.461076,0.357819,0.275241,0.498464,0.538463,0.19650,0.024909,0.319782,0.469907,0.040873,0.197817,0.585997,0.088080,0.101551,0.636993,0.708511,0.576

In [32]:
for col in model_df.columns:
    print(pd.Series(model_df[col].isna().value_counts()))

long
False    893442
Name: count, dtype: int64
lat
False    893442
Name: count, dtype: int64
year
False    893442
Name: count, dtype: int64
species
False    893442
Name: count, dtype: int64
vv_Descendingsummer
False    893442
Name: count, dtype: int64
vv_Descendingwinter
False    893442
Name: count, dtype: int64
vh_Descendingsummer
False    893442
Name: count, dtype: int64
vh_Descendingwinter
False    893442
Name: count, dtype: int64
vv_Ascendingsummer
False    893442
Name: count, dtype: int64
vv_Ascendingwinter
False    893442
Name: count, dtype: int64
vh_Ascendingsummer
False    893442
Name: count, dtype: int64
vh_Ascendingwinter
False    893442
Name: count, dtype: int64
VV_VH_Asummer
False    893442
Name: count, dtype: int64
VV_VH_Awinter
False    893442
Name: count, dtype: int64
VV_VH_Dsummer
False    893442
Name: count, dtype: int64
VV_VH_Dwinter
False    893442
Name: count, dtype: int64
VH_VV_Asummer
False    893442
Name: count, dtype: int64
VH_VV_Awinter
False    893442
Name: co

### Balanced Panel of Species

For seven species, (13,3,1,0,5,10,6) which have locations less than 5000, we have taken all the locations, and for the remaining 7, we have randomly sampled 5000 locations brininging the total observations to 64,147. This was the actual data frame with 532 features. But for ease of modelling we are restructuring the model frame a little but we will retain the sampling of 5000 locations per species in the actual frame. 

We merge the lat longs of the actual data frame (64,147 * 532) with our existing frame and then retain only 5000 locations per specie in case where we have more than 5000 locations in the data and if the data has locations less 5000, we keep all.

Here after grouping by Lat and Long, the number of rows equal the 148,907 which is our actual number of unique locations. After merge we expect this to be 64,147

In [33]:
model_df.groupby(['lat', 'long'])['species'].count().reset_index()

,lat,long,species
0,30.38513370000004,77.57655330000006,6
1,30.387641900000062,77.58138280000009,6
2,30.389346600000067,77.56340700000004,6
3,30.389401600000042,77.56335570000005,6
4,30.389493300000026,77.56316670000007,6
...,...,...,...
148902,32.94831170000003,76.45642000000004,6
148903,33.08003080000003,76.40363490000004,6
148904,33.08007490000006,76.40368610000007,6
148905,33.080129500000055,76.40363320000006,6


In [34]:
actual_df = pd.read_csv(actual_model_path,  dtype={"long": str, "lat": str})
actual_df

,long,lat,species,vv_Descendingsummer2018,vv_Descendingsummer2019,vv_Descendingsummer2020,vv_Descendingsummer2021,vv_Descendingsummer2022,vv_Descendingsummer2023,vv_Descendingwinter2018,vv_Descendingwinter2019,vv_Descendingwinter2020,vv_Descendingwinter2021,vv_Descendingwinter2022,vv_Descendingwinter2023,vh_Descendingsummer2018,vh_Descendingsummer2019,vh_Descendingsummer2020,vh_Descendingsummer2021,vh_Descendingsummer2022,vh_Descendingsummer2023,vh_Descendingwinter2018,vh_Descendingwinter2019,vh_Descendingwinter2020,vh_Descendingwinter2021,vh_Descendingwinter2022,vh_Descendingwinter2023,vv_Ascendingsummer2018,vv_Ascendingsummer2019,vv_Ascendingsummer2020,vv_Ascendingsummer2021,vv_Ascendingsummer2022,vv_Ascendingsummer2023,vv_Ascendingwinter2018,vv_Ascendingwinter2019,vv_Ascendingwinter2020,vv_Ascendingwinter2021,vv_Ascendingwinter2022,vv_Ascendingwinter2023,vh_Ascendingsummer2018,vh_Ascendingsummer2019,vh_Ascendingsummer2020,vh_Ascendingsummer2021,vh_Ascendingsummer2022,vh_Ascendingsummer2023,vh_Ascendingwinter2018,vh_Ascendingwinter2019,vh_Ascendingwinter2020,vh_Ascendingwinter2021,vh_Ascendingwinter2022,vh_Ascendingwinter2023,VV_VH_Asummer2018,VV_VH_Asummer2019,VV_VH_Asummer2020,VV_VH_Asummer2021,VV_VH_Asummer2022,VV_VH_Asummer2023,VV_VH_Awinter2018,VV_VH_Awinter2019,VV_VH_Awinter2020,VV_VH_Awinter2021,VV_VH_Awinter2022,VV_VH_Awinter2023,VV_VH_Dsummer2018,VV_VH_Dsummer2019,VV_VH_Dsummer2020,VV_VH_Dsummer2021,VV_VH_Dsummer2022,VV_VH_Dsummer2023,VV_VH_Dwinter2018,VV_VH_Dwinter2019,VV_VH_Dwinter2020,VV_VH_Dwinter2021,VV_VH_Dwinter2022,VV_VH_Dwinter2023,VH_VV_Asummer2018,VH_VV_Asummer2019,VH_VV_Asummer2020,VH_VV_Asummer2021,VH_VV_Asummer2022,VH_VV_Asummer2023,VH_VV_Awinter2018,VH_VV_Awinter2019,VH_VV_Awinter2020,VH_VV_Awinter2021,VH_VV_Awinter2022,VH_VV_Awinter2023,VH_VV_Dsummer2018,VH_VV_Dsummer2019,VH_VV_Dsummer2020,VH_VV_Dsummer2021,VH_VV_Dsummer2022,VH_VV_Dsummer2023,VH_VV_Dwinter2018,VH_VV_Dwinter2019,VH_VV_Dwinter2020,VH_VV_Dwinter2021,VH_VV_Dwinter2022,VH_VV_Dwinter2023,SAR_NDVI_Asummer2018,SAR_NDVI_Asummer2019,SAR_NDVI_Asummer2020,SAR_NDVI_Asummer2021,SAR_NDVI_Asummer2022,SAR_NDVI_Asummer2023,SAR_NDVI_Awinter2018,SAR_NDVI_Awinter2019,SAR_NDVI_Awinter2020,SAR_NDVI_Awinter2021,SAR_NDVI_Awinter2022,SAR_NDVI_Awinter2023,SAR_NDVI_Dsummer2018,SAR_NDVI_Dsummer2019,SAR_NDVI_Dsummer2020,SAR_NDVI_Dsummer2021,SAR_NDVI_Dsummer2022,SAR_NDVI_Dsummer2023,SAR_NDVI_Dwinter2018,SAR_NDVI_Dwinter2019,SAR_NDVI_Dwinter2020,SAR_NDVI_Dwinter2021,SAR_NDVI_Dwinter2022,SAR_NDVI_Dwinter2023,DVI_Asummer2018,DVI_Asummer2019,DVI_Asummer2020,DVI_Asummer2021,DVI_Asummer2022,DVI_Asummer2023,DVI_Awinter2018,DVI_Awinter2019,DVI_Awinter2020,DVI_Awinter2021,DVI_Awinter2022,DVI_Awinter2023,DVI_Dsummer2018,DVI_Dsummer2019,DVI_Dsummer2020,DVI_Dsummer2021,DVI_Dsummer2022,DVI_Dsummer2023,DVI_Dwinter2018,DVI_Dwinter2019,DVI_Dwinter2020,DVI_Dwinter2021,DVI_Dwinter2022,DVI_Dwinter2023,SVI_Asummer2018,SVI_Asummer2019,SVI_Asummer2020,SVI_Asummer2021,SVI_Asummer2022,SVI_Asummer2023,SVI_Awinter2018,SVI_Awinter2019,SVI_Awinter2020,SVI_Awinter2021,SVI_Awinter2022,SVI_Awinter2023,SVI_Dsummer2018,SVI_Dsummer2019,SVI_Dsummer2020,SVI_Dsummer2021,SVI_Dsummer2022,SVI_Dsummer2023,SVI_Dwinter2018,SVI_Dwinter2019,SVI_Dwinter2020,SVI_Dwinter2021,SVI_Dwinter2022,SVI_Dwinter2023,RDVI_Asummer2018,RDVI_Asummer2019,RDVI_Asummer2020,RDVI_Asummer2021,RDVI_Asummer2022,RDVI_Asummer2023,RDVI_Awinter2018,RDVI_Awinter2019,RDVI_Awinter2020,RDVI_Awinter2021,RDVI_Awinter2022,RDVI_Awinter2023,RDVI_Dsummer2018,RDVI_Dsummer2019,RDVI_Dsummer2020,RDVI_Dsummer2021,RDVI_Dsummer2022,RDVI_Dsummer2023,RDVI_Dwinter2018,RDVI_Dwinter2019,RDVI_Dwinter2020,RDVI_Dwinter2021,RDVI_Dwinter2022,RDVI_Dwinter2023,NRDVI_Asummer2018,NRDVI_Asummer2019,NRDVI_Asummer2020,NRDVI_Asummer2021,NRDVI_Asummer2022,NRDVI_Asummer2023,NRDVI_Awinter2018,NRDVI_Awinter2019,NRDVI_Awinter2020,NRDVI_Awinter2021,NRDVI_Awinter2022,NRDVI_Awinter2023,NRDVI_Dsummer2018,NRDVI_Dsummer2019,NRDVI_Dsummer2020,NRDVI_Dsummer2021,NRDVI_Dsummer2022,NRDV

In [35]:
species_sampled = pd.merge(model_df, actual_df[['long', 'lat']], on=['long', 'lat'], how='inner', validate='m:1')
species_sampled.groupby(['lat', 'long'])['species'].count().reset_index()

,lat,long,species
0,30.38513370000004,77.57655330000006,6
1,30.387641900000062,77.58138280000009,6
2,30.389401600000042,77.56335570000005,6
3,30.389493300000026,77.56316670000007,6
4,30.38954030000008,77.56328840000003,6
...,...,...,...
64142,32.94321140000005,76.46184580000005,6
64143,32.943245700000034,76.17807740000006,6
64144,32.94332000000003,76.17807380000005,6
64145,32.948300000000074,76.45644330000005,6


The number of rows above is now 64,147 during groupby using Lat and Long.

### Checking for species count
The following process ensures that we have the same number per species after processing.

In [36]:
unique_loc = species_sampled.groupby(['lat', 'long'])['species'].count().reset_index().drop('species', axis=1)
unique_loc.merge(actual_df[['long', 'lat', 'species']], on=['long', 'lat'], how='outer')['species'].value_counts()


species
4     5000
11    5000
7     5000
12    5000
2     5000
8     5000
9     5000
6     4834
10    4765
5     4737
0     4206
1     3939
3     3524
13    3142
Name: count, dtype: int64

### Frame to be considered for modelling

In [37]:
species_sampled


,long,lat,year,species,vv_Descendingsummer,vv_Descendingwinter,vh_Descendingsummer,vh_Descendingwinter,vv_Ascendingsummer,vv_Ascendingwinter,vh_Ascendingsummer,vh_Ascendingwinter,VV_VH_Asummer,VV_VH_Awinter,VV_VH_Dsummer,VV_VH_Dwinter,VH_VV_Asummer,VH_VV_Awinter,VH_VV_Dsummer,VH_VV_Dwinter,SAR_NDVI_Asummer,SAR_NDVI_Awinter,SAR_NDVI_Dsummer,SAR_NDVI_Dwinter,DVI_Asummer,DVI_Awinter,DVI_Dsummer,DVI_Dwinter,SVI_Asummer,SVI_Awinter,SVI_Dsummer,SVI_Dwinter,RDVI_Asummer,RDVI_Awinter,RDVI_Dsummer,RDVI_Dwinter,NRDVI_Asummer,NRDVI_Awinter,NRDVI_Dsummer,NRDVI_Dwinter,SSDVI_Asummer,SSDVI_Awinter,SSDVI_Dsummer,SSDVI_Dwinter,B11summer,B11winter,B02summer,B02winter,B12summer,B12winter,B08summer,B08winter,B04summer,B04winter,B03summer,B03winter,NDVIsummer,NDVIwinter,ARVI2summer,ARVI2winter,BWDRVIsummer,BWDRVIwinter,CVIsummer,CVIwinter,CTVIsummer,CTVIwinter,EVI2summer,EVI2winter,GVMIsummer,GVMIwinter,MSVAIhypersummer,MSVAIhyperwinter,MTVI2summer,MTVI2winter,MNDVIsummer,MNDVIwinter,OSAVIsummer,OSAVIwinter,PVIsummer,PVIwinter,SARVIsummer,SARVIwinter,SLAVIsummer,SLAVIwinter,TSAVI2summer,TSAVI2winter,WDVIsummer,WDVIwinter,WDRVIsummer,WDRVIwinter,sand_f,elevation,cfvo_f,phh2o_f,soc_f,slope,clay_f,nitrogen_f,cec_f,bdod_f,aspect,silt_f,ocd_f
0,75.66669580000007,32.25107380000003,2018,1,0.396606,0.471055,0.416042,0.470285,0.451115,0.404824,0.341066,0.350320,0.338847,0.269886,0.336448,0.307829,0.519535,0.602601,0.497106,0.506712,0.584585,0.662452,0.574248,0.594059,0.621240,0.716986,0.636023,0.628458,0.392045,0.417418,0.344660,0.484055,0.605552,0.680172,0.596939,0.617232,0.564211,0.645100,0.552461,0.572055,0.664566,0.754933,0.690628,0.665937,0.313081,0.228517,0.162219,0.118167,0.317432,0.262492,0.310292,0.278019,0.190256,0.129642,0.190458,0.140687,0.433215,0.551315,0.433215,0.551315,0.487001,0.646174,0.005272,0.280977,0.526269,0.633711,0.382663,0.421141,0.431660,0.561714,0.408945,0.475636,0.363129,0.447514,0.412795,0.542756,0.493089,0.560768,0.310292,0.278019,1.0,1.0,0.114210,0.146335,0.461660,0.460997,0.303479,0.271846,0.433215,0.551315,0.19650,0.024909,0.319782,0.469907,0.040873,0.197817,0.585997,0.088080,0.101551,0.636993,0.708511,0.576992,0.118323
1,75.66669580000007,32.25107380000003,2019,1,0.460550,0.446541,0.429824,0.487266,0.366455,0.478185,0.370159,0.362405,0.183898,0.322430,0.383513,0.300665,0.709946,0.540842,0.444008,0.515246,0.761503,0.603882,0.522477,0.602234,0.788002,0.664917,0.574423,0.633923,0.355852,0.417348,0.407194,0.471022,0.774423,0.624019,0.547459,0.624950,0.748836,0.584238,0.498555,0.580663,0.808862,0.710048,0.625380,0.669283,0.274354,0.238709,0.117719,0.122591,0.257691,0.263396,0.272727,0.308818,0.142307,0.134439,0.137201,0.143900,0.533531,0.565906,0.533531,0.565906,0.580706,0.665711,0.008468,0.302683,0.619712,0.646971,0.433430,0.450585,0.473029,0.584626,0.457725,0.509849,0.404839,0.477641,0.441885,0.570223,0.577584,0.584228,0.272727,0.308818,1.0,1.0,0.150823,0.158899,0.461616,0.461170,0.312718,0.302638,0.533531,0.565906,0.19650,0.024909,0.319782,0.469907,0.040873,0.197817,0.585997,0.088080,0.101551,0.636993,0.708511,0.576992,0.118323
2,75.66669580000007,32.25107380000003,2020,1,0.431712,0.452766,0.498169,0.413335,0.378641,0.409808,0.347632,0.350206,0.238847,0.333536,0.381426,0.385602,0.638601,0.528305,0.446605,0.421169,0.696887,0.591767,0.524909,0.509017,0.731041,0.645299,0.569751,0.561549,0.350470,0.365744,0.410065,0.458816,0.713037,0.612352,0.549750,0.536319,0.681090,0.571704,0.501122,0.483209,0.762694,0.695944,0.626819,0.614591,0.296447,0.233513,0.151789,0.112627,0.289345,0.264374,0.335080,0.286403,0.173430,0.134302,0.177465,0.139276,0.498464,0.538463,0.498464,0.538463,0.514843,0.666735,0.005749,0.296927,0.588618,0.621737,0.451147,0.416834,0.484453,0.555216,0.479883,0.471391,0.434758,0.435182,0.469696,0.537254,0.569279,0.550948,0.335080,0.286403,1.0,1.0,0.149061,0.141805,0.461722,0.461076,0.357819,0.275241,0.498464,0.538463,0.19650,0.024909,0.319782,0.469907,0.040873,0.197817,0.585997,0.088080,0.101551,0.636993,0.708511,0.576

### Exporting File

In [38]:
species_sampled.to_parquet(model_df_path, index=False)
# model_df.to_parquet(species_model_df_path, index=False)

In [39]:
# species_sampled[species_sampled["species"].isin([10,8])]